In [1]:
!pip install cassandra-driver clickhouse-driver pandas pyspark

  Using cached cassandra_driver-3.29.3-cp313-cp313-win_amd64.whl.metadata (7.0 kB)
  Using cached clickhouse_driver-0.2.10-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached pandas-3.0.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pyspark-4.1.1.tar.gz (455.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached geomet-1.1.0-py3-none-any.whl.metadata (11 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached numpy-2.4.1-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [1925 lines of output]
  running bdist_wheel
  C:\Users\juanh\AppData\Local\Temp\pip-build-env-zbpcbxp0\overlay\Lib\site-packages\setuptools\_distutils\cmd.py:135: SetuptoolsDeprecationWarning: bdist_wheel.universal is deprecated
  !!
  
          ********************************************************************************
          With Python 2.7 end-of-life, support for building universal wheels
          (i.e., wheels that support both Python 2 and Python 3)
          is being obviated.
          Please discontinue using this option, or if you still need it,
          file an issue with pypa/setuptools describing your use case.
  
          This deprecation is overdue, please update your project and remove deprecated
          calls to avoid build errors in the future.
          ********************************************************************************
  
  !!
    self.finalize_options()
  running build
  runnin

In [3]:
from cassandra.cluster import Cluster
import random

# Conexión a Cassandra (Docker)
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Crear espacio de trabajo y tabla
session.execute("CREATE KEYSPACE IF NOT EXISTS proyecto_bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor':1};")
session.set_keyspace('proyecto_bigdata')
session.execute("CREATE TABLE IF NOT EXISTS ventas_crudas (id int PRIMARY KEY, categoria text, precio float, fecha timestamp);")

# Insertar 100,000 registros
print("Cargando datos en Cassandra...")
for i in range(100000):
    cat = random.choice(['Electronica', 'Ropa', 'Hogar', 'Juguetes'])
    precio = random.uniform(5.0, 1000.0)
    session.execute(f"INSERT INTO ventas_crudas (id, categoria, precio, fecha) VALUES ({i}, '{cat}', {precio}, toTimestamp(now()))")

print("✅ 100,000 registros cargados en la tabla 'ventas_crudas'.")


ModuleNotFoundError: No module named 'cassandra'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("ETL_UNEG") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .getOrCreate()

# Leer de Cassandra
df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
    .load()

# Transformación: Agrupar por categoría y sumar
ventas_resumen = df.groupBy("categoria").agg(
    F.sum("precio").alias("total_ventas"),
    F.count("id").alias("cantidad_transacciones")
)

ventas_resumen.show()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
from clickhouse_driver import Client

client = Client(host='localhost')
client.execute('CREATE TABLE IF NOT EXISTS ventas_resumen (categoria String, total_ventas Float64, cantidad_transacciones UInt64) ENGINE = Memory')

# Convertir Spark DF a Pandas para subirlo a ClickHouse fácilmente
pandas_df = ventas_resumen.toPandas()
client.execute('INSERT INTO ventas_resumen VALUES', pandas_df.to_dict('records'))

print("✅ Datos cargados con éxito en la tabla 'ventas_resumen' de ClickHouse.")